# Traffic Vision: Multi-Camera Recording System

## Objective
Develop continuous multi-camera GDOT traffic stream recording for ML training data.

## Technical Approach
- Streamlink + FFmpeg pipeline
- CSV-driven camera configuration
- Parallel recording processes
- Systemd service for 24/7 operation

**Date:** June 8, 2025  
**Branch:** `experiment/multi-camera-recording`

In [20]:
import pandas as pd

# load camera data
df = pd.read_csv('../data/gdot_midtown_atlanta_streams.csv')
print(f"cameras: {len(df)}")
print(f"columns: {list(df.columns)}")
print(df.head())

cameras: 8
columns: ['Date Added', 'Camera_ID', 'Location', 'Page', 'Stream_URL']
   Date Added Camera_ID                               Location  Page  \
0  2025-04-11  ATL-0610         10th St at Monroe Dr (Atlanta)     6   
1  2025-04-11  ATL-0972       Peachtree St at 5th St (Atlanta)    15   
2  2025-04-11  ATL-1005      Peachtree St at 12th St (Atlanta)    15   
3  2025-04-11  ATL-0907      Piedmont Ave at 14th St (Atlanta)    15   
4  2025-04-11  ATL-0997  West Peachtree St at 5th St (Atlanta)    27   

                                          Stream_URL  
0  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
1  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
2  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
3  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
4  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  


In [21]:
import subprocess
import json
import glob
import os

# analyze sample recordings
recording_files = glob.glob('../sample_recordings/*/*.mp4')
print(f"recordings found: {len(recording_files)}")

for file in recording_files[:3]:
    size = os.path.getsize(file)
    
    # get metadata
    cmd = ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '-show_streams', file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        data = json.loads(result.stdout)
        duration = float(data['format']['duration'])
        video_stream = next((s for s in data['streams'] if s['codec_type'] == 'video'), None)
        
        if video_stream:
            fps_str = video_stream['r_frame_rate']
            fps = float(fps_str.split('/')[0]) / float(fps_str.split('/')[1]) if '/' in fps_str else float(fps_str)
            
            print(f"{file}: {size:,} bytes, {duration:.1f}s, {video_stream['width']}x{video_stream['height']}, {fps:.1f}fps")

recordings found: 4
../sample_recordings/ATL-0972/ATL-0972_20250608_201300.mp4: 100,749 bytes, 16.5s, 480x270, 15.0fps
../sample_recordings/ATL-0972/ATL-0972_20250608_202414.mp4: 129,784 bytes, 16.5s, 480x270, 15.0fps
../sample_recordings/ATL-0610/ATL-0610_20250608_201300.mp4: 248,641 bytes, 20.7s, 480x270, 15.0fps


## Results
- 7 cameras recording simultaneously
- 480x270 resolution, 15fps, h264 codec
- 15-second test segments: ~100-250KB each
- Deployed as systemd service on HPC workstation
- Production uses 15-minute segments for training data

## Architecture
```mermaid
graph TD
    A[GDOT Cameras] --> B[HLS Streams]
    B --> C[HPC Workstation]
    C --> D[Parallel Recording]
    D --> E[Local Storage]
    E --> F[YOLO Training Data]